# Normas de vetores e matrizes

Existem 3 tipos de normas: Norma euclidiana ($||X||_E$), Norma infinita ($||X||_\infty$) e Norma "um" ($||X||_1$)

### Norma Euclidiana ($||X||_E$)

É a norma mais comum de todas, que já estamos acostumados a fazer. Em um vetor, você soma os quadrados de todos os membros e tira a raiz quadrada do resultado, isto é:

$||X||_{E} = \sqrt{\sum_{i=1}^{n} X_i ^ 2}$

Sendo $n$ o tamanho do vetor.

obs: ele só passou a versão para vetores dessa norma

In [1]:
import numpy as np

np.linalg.norm([1,1,1])

1.7320508075688772

### Norma infinita (ou norma do máximo) ($||X||_\infty$)

Em um vetor, essa norma é simplesmente o maior membro do vetor em módulo:

$||A||_\infty = max_{(1 \leq i \leq n)}(|X_i|)$



Já, no caso de uma matriz $A$, a norma é o somatório da linha de maior somatório (somando os números em módulo), isto é:

$||A||_\infty = max_{(1 \leq i \leq n)}(\sum_{j=1}^{n} |X_{ij}|) = max(\sum_{j=1}^{n} |X_{1j}| , \sum_{j=1}^{n} |X_{2j}| , ... , \sum_{j=1}^{n} |X_{nj}|) $

In [ ]:
import numpy as np

def infiniteNorm(a):
    try:
        # Retornamos o maximo das somas das colunas.
        return np.max([np.sum(a[i, :]) for i in range(a.shape[1])])
    except(IndexError):
        return np.max(a)

### Norma "um" ($||X||_1$)

Em um vetor, essa norma é simplesmente a soma de todos os elementos (em módulo):

$||A||_1 = \sum_{i=1}^{n} |X_i| $

Já, no caso de uma matriz $A$, a norma é equivalente à norma infinita, porém, ao invés de ser a soma das linhas, é a soma das colunas, ou seja:

$||A||_1 = max_{(1 \leq j \leq n)}(\sum_{i=1}^{n} |X_{ij}|) = max(\sum_{i=1}^{n} |X_{i1}| , \sum_{i=1}^{n} |X_{i2}| , ... , \sum_{i=1}^{n} |X_{in}|) $

In [36]:
import numpy as np

def manhattan(a):
    try:
        # Retornamos o maximo das somas das colunas.
        return np.max([np.sum(a[: , i]) for i in range(a.shape[1])])
    except(IndexError):
        return np.sum(a)

In [37]:
manhattan(np.array([[1, 2, 3],
                    [4, 5, 6],
                    [7, 8, 9]]))

18

# Resolução de sistemas lineares

## Método de eliminação de Gauss

O método de elmininação de Gauss consiste na transformação de um sistema linear de forma $Ax=b$ em um sistema equivalente $A^{n}x=b^{n}$ por meio da realização de operações elementares nas linhas da matriz aumentada $[A|b]$

A resolução completa de um sistema $NxN$ é realizada em $N-1$ passos.

### Algoritmo completo

1. Montar a matriz aumentada $[A|b]$
2. Determinação do pivô: $a_{kk}$
3. Definir os multiplicadores de linha: $m_{ik} = \frac{a_{ik}}{a_{kk}}$
4. Atualização das linhas: $L_i \leftarrow L_i - m_{ik} \times L_{pivo}$

<span style="color:red">Importante: devido ao Python, o código abaixo pode produzir sistemas lineares resolvidos com erros na base de 10^{-10}, o que deve ser considerado durante a análise da resposta obtida. Além disso, o código deve receber como _input_ uma matriz pivôs nulos.</span>

In [ ]:
import numpy as np


"""
    Codigo que resolve sistemas lineares. Por uma questao 
    de simplificacao de implementacao, nao serao tratados
    os casos em que os pivos sao 0s, logo cabe ao usuario 
    realizar um tratamento da entrada.
"""
def metodoGauss(A, b):
    
    # A partir da concatenacao das matrizes, montamos a [A|b]
    matrix = np.concatenate((A, b), axis=1)
    matrix.astype("float64")
    for i in range(matrix.shape[0] - 1):
         
        # Calculo dos multiplicadores    
        print(">>>Matrix:")
        print(matrix)
        
        multiplicadores = matrix[:, i]/matrix[i][i]
        print(">>>Multiplicadores: ")
        print(multiplicadores)
        # Atualizamos as linhas abaixo da linha atual...         
        for j in range(i + 1, matrix.shape[0]):
            print("Produto atual: ", matrix[j], multiplicadores[j],matrix[i] )
            matrix[j] = matrix[j] - multiplicadores[j] * matrix[i]
        
    return matrix

In [ ]:
metodoGauss(np.array([[3.0, 2., 4.],
                      [1.0, 1., 2.],
                      [4.0, 3., -2.]]),
            np.array([[1.], 
                      [2.],
                      [3.]]))

## Métodos de fatoração LU

Os métodos de fatoração LU consistem na ideia de que existe uma matriz $L$ triangular inferior e uma matriz $U$ triangular superior, tais que:

$LU = A$

Assim, dado o sistema linear $AX = b$, temos:

$LU = A \implies (LU)X=b \implies L(UX) = b$

Fazendo $UX = Y$, obtemos o sistema linear triangular $LY = b$.

A ideia é, basicamente, resolver o sistema:

$\begin{cases} LY = b \\ UX = Y \end{cases}$

## Método de DooLittle (LU)

#### 1º passo: calcular $L$ e $U$:

Atribuindo $1$ para todos os valores da diagonal da matriz $L$, possuímos um sistema de $n^2$ equações e $n^2$ incógnitas.

Em seguida, efetuamos o produto $LU$ e igualamos à matriz $A$. Assim, temos que a primeira linha de $U$ é idêntica à primeira linha de $A$, enquanto a primeira coluna de $L$ é dada por:

$l_{k1}u_{11} = a_{k1} \implies l_{k1} = \frac{a_{k1}}{a_{11}}$, <br><br> com $k$ variando de $2$ a $n$
(lembrando que $u_{11} = a_{11}$)

Agora que temos a primeira linha de $U$ e a primeira coluna de $L$, podemos seguir com as equações:

##### Linha $k$ de $U$:
$u_{kj} = a_{kj} - \sum_{m=1}^{k-1} l_{km}u_{mj}$, $j$ variando de $k$ a $n$.

##### Coluna $k$ de $L$: 
$l_{ik} = a_{ik} - \frac{\sum_{m=1}^{k-1} l_{im}u_{mk}}{u_{kk}} $,  $i$ variando de $k+1$ a $n$.

Obs: Calculamos primeiro a linha $k$ de $U$, depois a coluna $k$ de $L$, incrementamos $k$, e repetimos o algoritmo até que $k = n$

#### 2º passo: resolver o sistema acima

Resolve-se primeiro $LY = b$, obtendo-se $Y$ <br>
Resolve-se depois $UX = Y$, obtendo-se $X$

Como as matrizes são triangulares, é bem fácil de resolver o sistema, pois $y_1 = \frac{b_1}{l_{11}}$ e $x_n = \frac{y_n}{u_{nn}}$

## Método de Cholesky

Caso especial em que $A$ é uma matriz simétrica. Temos que $U = L^t$